In [ ]:
import sys
from wbib import wbib
import requests
from pybtex.database.input import bibtex
from tqdm import tqdm
import time

def get_qid_for_title(title):
    """
    Gets the best Wikidata candidate from the title of the paper. 
    """
    api_call = f"https://www.wikidata.org/w/api.php?action=wbsearchentities&search={title}&language=en&format=json"
    api_result = requests.get(api_call).json()
    if api_result["success"] == 1:
        return(api_result["search"][0]["id"])


parser = bibtex.Parser()
bibtexFile= parser.parse_file(sys.argv[1])   

list_of_qids = []
print("===== Querying Wikidata to get IDs =====")
with open("error_log.txt", "w") as f:
    f.write("Titles that were not found on Wikidata: \n")
    for i in tqdm(bibtexFile.entries):
        title = bibtexFile.entries[i].fields["title"]
        title = title.replace("{", "")
        title = title.replace("}", "")
        try:
            list_of_qids.append(get_qid_for_title(title))
        except:
            tqdm.write(f"Failed for {title}")
            f.write(f"{title}\n")
        time.sleep(0.3)

with open("index.html", "w") as f:
    html = wbib.render_dashboard(list_of_qids)
    f.write(html)